# AI Agents from First Principles
> Following along Pramod Goyal's excellent blog [here](https://goyalpramod.github.io/blogs/AI_agents_from_first_principles/). I'll be using OpenRouter for API calls. This file should serve as a decent mostly-code reference. 

## Tools
This has a needlessly complex name - **tools are just functions.**

Yes, that’s it - they are functions with defined inputs and outputs. These functions are provided to an LLM as a schema, and the model extracts input values from user queries to call these functions.

Here's a sample tool

In [1]:
def get_weather(city: str, date: str | None = None) -> dict[str, int | str]:
    """
    Gets weather information for a specific city and date.#-

    Args:
        city: Name of the city#-
        date: Date in YYYY-MM-DD format, defaults to today#-

    Returns:
        dict: Weather information including temperature and conditions#-
    """
    # Implementation here
    return {"temperature": 25, "conditions": "sunny", "humidity": 60}


> `Callable[..., ...]` is not allowed it seems, you have to use `Callable[..., Any]`

In many libraries you will find them using `@tool` on top of functions. This is **just a Python decorator that adds metadata.** Let’s create a simple one:

In [2]:
from functools import wraps
from typing import Callable, Any, Dict
import inspect


# This is the decorator that we define
def tool(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Decorator that converts a function into an LLM tool by adding metadata.

    Args:
        func Callable[..., Any]: Function to convert into a tool

    Returns:
        Callable[..., Any]: Decorated function with metadata
    """

    @wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        return func(*args, **kwargs)

    # Add function metadata
    wrapper.is_tool: bool = True  # type: ignore
    wrapper.description: str = func.__doc__ if func.__doc__ else ""  # type: ignore
    wrapper.parameters: inspect.Signature = inspect.signature(func).parameters  # type: ignore

    return wrapper


# Usage example
@tool
def calculate_area(length: float, width: float) -> float:
    """Calculate area of a rectangle."""
    return length * width


# Let's see what the decorator added
print(
    "Is tool:",
    calculate_area.is_tool,  # type: ignore
)  # Is tool: True

print(
    "Description:",
    calculate_area.description,  # type: ignore
)  # Description: Calculate area of a rectangle.

print(
    "Parameters:",
    calculate_area.parameters,  # type: ignore
)  # Parameters: <Signature (length: float, width: float) -> float>
print("Result:", calculate_area(5, 3))  # result: 15


Is tool: True
Description: Calculate area of a rectangle.
Parameters: OrderedDict([('length', <Parameter "length: float">), ('width', <Parameter "width: float">)])
Result: 15


The **decorator doesn’t modify how the function works** - it simply passes through all calls to the original function while **adding extra attributes**. These attributes (`is_tool`, `description`, `parameters`) help the LLM understand:

- That this function is available as a tool
- What the function does (from the docstring)
- What parameters it expects (from the signature)

This metadata is then used to create the function schema that we send to the LLM.



## Memory

Memory in LLM agents can be handled in two ways - through context window or external databases. Here is a basic example -

```python
import datetime

class AgentMemory:
    def __init__(self):
        """Initialize memory storage"""
        self.conversations = []  # Short-term memory
        self.db = Database()   # Long-term memory

    def add_to_memory(self, message: str, role: str):
        """Add a new message to memory"""
        # Keep last N messages in context
        self.conversations.append(
            {"role": role, "content": message, "timestamp": datetime.now()}
        )

        # Store in long-term memory
        self.db.store(message, role)

    def get_relevant_context(self, query: str) -> List[str]:
        """Retrieve relevant information from long-term memory"""
        return self.db.search(query)

    def get_context_window(self) -> List[dict]:
        """Get recent conversations for context window"""
        return self.conversations[-10:]  # Last 10 messages
```

**Here’s a basic RAG implementation:**

```python
from sentence_transformers import SentenceTransformer 
from qdrant_client import QdrantClient 
from typing import List, Dict

class RAGSystem:
    def __init__(self):
        # Initialize embedding model
        self.embedder = SentenceTransformer('BAAI/bge-large-en-v1.5')
        
        # Initialize vector store
        self.qdrant: QdrantClient = QdrantClient("localhost", port=6333)
        
    def add_documents(self, documents: List[str]):
        # Create embeddings
        embeddings = self.embedder.encode(documents)
        
        # Store in vector DB
        self.qdrant.upload_collection(
            collection_name="knowledge_base",
            vectors=embeddings,
            payload=documents
        )
    
    def retrieve(self, query: str, k: int = 3) -> List[str]:
        # Get query embedding
        query_embedding = self.embedder.encode(query)
        
        # Search similar documents
        results = self.qdrant.search(
            collection_name="knowledge_base",
            query_vector=query_embedding,
            limit=k
        )
        return [hit.payload for hit in results]
```



## Document Processing


In [3]:
from typing import List
import re


def chunk_text(text: str, chunk_size: int = 5) -> List[str]:
    """
    Split text into smaller chunks while preserving sentence boundaries
    """
    sentences = text.split(". ")
    chunks: List[str] = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        if current_length + len(sentence) > chunk_size:
            # if new is bigger, then add existing to chunks. Then make new as current
            chunks.append(" ".join(current_chunk))
            current_chunk = [sentence]
            current_length = len(sentence)
        else:
            current_chunk.append(sentence)
            current_length += len(sentence)

    return chunks


def clean_text(text: str) -> str:
    """
    Standardize text format and remove noise
    """
    # Remove extra whitespace
    text = " ".join(text.split())
    # Remove special characters
    text = re.sub(r"[^\w\s.,!?]", "", text)
    return text


In [4]:
chunk_text(
    "This is a long sentence. It should be split into smaller chunks. My already existing persona needs slight modification such that there should me higher interest in tyres and robotics. "
)

['',
 'This is a long sentence',
 'It should be split into smaller chunks',
 'My already existing persona needs slight modification such that there should me higher interest in tyres and robotics']

## Best Practices for AI Agents

### Core Guidelines
As pointed out in the [smolagents guide](https://huggingface.co/docs/smolagents/tutorials/building_good_agents), minimize LLM calls by:

- Combining related tools into single functions
- Using deterministic logic over LLM-based decisions
- Caching responses for similar queries


```python
# Bad: Multiple LLM calls
def process_order(order_details):
    # First LLM call to validate
    validated = llm.validate(order_details)
    if not validated:
        return "Invalid order"
        
    # Second LLM call to format
    formatted = llm.format(order_details)
    
    # Third LLM call to process
    return llm.process(formatted)

# Good: Single LLM call
def process_order(order_details):
    # Validate with regular code
    if not is_valid_order(order_details):
        return "Invalid order"
        
    # Format with template
    formatted = ORDER_TEMPLATE.format(**order_details)
    
    # Single LLM call
    return llm.process(formatted)

```


### Common Pitfalls

**Over-relying on LLMs**

Bad:

```python
def validate_email(email):
    response = llm.call("Is this a valid email: " + email)
    return "valid" in response.lower()

```


Good:

```python
import re

def validate_email(email):
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+
```
---

**Prompt Bloat**

Bad:

```python
system_message = """You are an AI assistant that helps with tasks.
You should be helpful, concise, and clear.
Always format responses properly.
Remember to be polite.
Double check your answers.
Consider edge cases.
...50 more lines of instructions..."""

```

Good:

```python
system_message = """You are a task-focused AI assistant.
Format: Generate concise, actionable responses.
Priority: Accuracy and clarity."""

```
---

**Tool Proliferation**

Instead of:

```py
tools = [
    fetch_weather,
    fetch_temperature,
    fetch_humidity,
    fetch_wind_speed,
    fetch_precipitation
]

```


Do this:

```py
tools = [
    fetch_weather_data  # Returns complete weather info in one call
]

```
---


**Testing Framework**

```py
class AgentTester:
    def __init__(self, agent):
        self.agent = agent
        self.metrics = {
            'success_rate': 0,
            'response_time': [],
            'token_usage': [],
            'error_rate': 0
        }
    
    def test_case(self, input_query, expected_output):
        start_time = time.time()
        try:
            response = self.agent.run(input_query)
            success = self.validate_response(response, expected_output)
            self.metrics['success_rate'] += success
            
        except Exception as e:
            self.metrics['error_rate'] += 1
            
        self.metrics['response_time'].append(time.time() - start_time)
        
    def validate_response(self, response, expected):
        # Implement validation logic
        pass
        
    def get_metrics_report(self):
        return {
            'avg_response_time': np.mean(self.metrics['response_time']),
            'success_rate': self.metrics['success_rate'],
            'error_rate': self.metrics['error_rate']
        }

```


# Building an Agent
<img src="readme-images/1.png" alt="drawing" width="1000"/>

### Creating OpenAI Client

In [5]:
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()

openrouter_api_key = os.getenv("OPENROUTER_API_KEY")

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=openrouter_api_key,
)

completion = client.chat.completions.create(
    model="google/gemma-3-1b-it:free",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What is your name?"},
            ],
        }
    ],
)
print(completion.choices[0].message.content)

 Don’t worry, you don’t need to! I’m Gemma, a large language model created by the Gemma team at Google DeepMind.


In [6]:
type(completion)

openai.types.chat.chat_completion.ChatCompletion

### LLM call

> I tried to use the free `"google/gemma-3-1b-it:free"`, but it does not have tool calling abilities it seems. `openai/gpt-3.5-turbo-0613` does not support tool calling as well. `o3-mini` and `4o` models seem to perform the same

In [7]:
from typing import Optional
from openai.types.chat.chat_completion import ChatCompletion
from openai.types.chat.chat_completion_message import ChatCompletionMessage


def run_llm(
    content: Optional[str] = None,
    messages: Optional[List[str]] = [],
    # tool_schemas: Optional[List[str]] = [],#-
    tool_schemas: Optional[List[Dict[str, Any]]] = [],
    system_message: str = "You are a helpful assistant.",
) -> List[ChatCompletionMessage]:
    # Build base request parameters
    request_params: Dict[str, Any] = {
        "model": "openai/o3-mini",  # "openai/gpt-4o-2024-11-20",  # openai/gpt-3.5-turbo-0613", # ,
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": content},
        ]
        + messages,  # type: ignore
    }

    # Only add tools parameter if tool_schemas is provided and non-empty
    if tool_schemas:
        request_params["tools"] = tool_schemas

    # Make the API call with conditional parameters
    completion: ChatCompletion = client.chat.completions.create(**request_params)  # type: ignore

    response: Dict[str, Any] = completion.choices[0].message  # type: ignore
    messages.append(response)  # type: ignore

    return messages


# {"conversationId":"cf746cdd-b8fc-498a-b3e2-fd84a89c4cbf","source":"instruct"}



Let’s give it the same prompt that we made earlier and see how it works.


In [8]:
run_llm(
    content="""["apple", "pie", 42, 2, 13]""",
    system_message="""
    You are an expert classifier, which classifies strings and integers.\
    Given a list of numbers and words, only return the numbers as a list.\
    You will be given the inputs inside <input> tags.\

    Input:  <input>["hello", 42, "pizza", 2, 5]</input>
    Output: [42,2,5]
    """,
)

# [ChatCompletionMessage(content='[42, 2, 13]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)]


[ChatCompletionMessage(content='[42,2,13]', role='assistant', function_call=None, tool_calls=None, refusal=None)]

This works as expected, Let’s build on top of this by giving our LLM the ability to calculate sums of numbers. (You can define the function anyhow you would like)


### LLM call + Tools

**Let’s create a utility function that takes another function and creates it’s schema**

In [9]:
from openai.types.chat import ChatCompletionToolParam


# def function_to_schema(func: Callable[..., Any]) -> Dict[str, Any]:
def function_to_schema(func: Callable[..., Any]) -> ChatCompletionToolParam:
    type_map: Dict[Any, str] = {
        str: "string",
        int: "integer",
        float: "number",
        bool: "boolean",
        list: "array",
        dict: "object",
        type(None): "null",
    }

    try:
        # Get the function's signature
        signature = inspect.signature(func)
    except ValueError as e:
        raise ValueError(
            f"Failed to get signature for function {func.__name__}: {str(e)}"
        )

    # Define the parameters dictionary
    parameters: Dict[str, str | Dict[str, str]] = {}
    for param in signature.parameters.values():
        try:
            param_type = type_map.get(param.annotation, "string")
        except KeyError as e:
            raise KeyError(
                f"Unknown type annotation {param.annotation} for parameter {param.name}: {str(e)}"
            )
        parameters[param.name] = {"type": param_type}

    # No default value means required
    required = [
        param.name
        for param in signature.parameters.values()
        if param.default == inspect._empty  # type: ignore
    ]

    # Return the schema dictionary
    return {
        "type": "function",
        "function": {
            "name": func.__name__,
            "description": (func.__doc__ or "").strip(),
            "parameters": {
                "type": "object",
                "properties": parameters,
                "required": required,
            },
        },
    }

**A function that takes another function can be simplified using a decorator. That is what one usually sees in most libraries/framework “@tool”. Now let’s create a sum tool.**


In [10]:
import json


def add_numbers(num_list: List[int]):
    """
    This function takes a List of numbers as Input and returns the sum

    Args:
        input: List[int]
        output: int
    """
    return sum(num_list)


schema = function_to_schema(add_numbers)
print(json.dumps(schema, indent=2))

# {
#   "type": "function",
#   "function": {
#     "name": "add_numbers",
#     "description": "This function takes a List of numbers as Input and returns the sum \n  \n  Args:\n      input: List[int]\n      output: int",
#     "parameters": {
#       "type": "object",
#       "properties": {
#         "num_list": {
#           "type": "string"
#         }
#       },
#       "required": [
#         "num_list"
#       ]
#     }
#   }
# }


{
  "type": "function",
  "function": {
    "name": "add_numbers",
    "description": "This function takes a List of numbers as Input and returns the sum\n\n    Args:\n        input: List[int]\n        output: int",
    "parameters": {
      "type": "object",
      "properties": {
        "num_list": {
          "type": "string"
        }
      },
      "required": [
        "num_list"
      ]
    }
  }
}


In [11]:
type(schema)

dict

More dummy proof:

In [12]:
from typing import List, Union
import ast


def add_numbers(num_list: Union[List[int], str]) -> int:
    """
    This function takes either a List of integers or a string representation of a list
    and returns the sum of the numbers.

    Args:
        num_list: List[int] or str - Either a list of integers or a string representing a list
            e.g. "[1, 2, 3]" or [1, 2, 3]

    Returns:
        int: The sum of all numbers in the list

    Raises:
        ValueError: If the string cannot be converted to a list of integers
        SyntaxError: If the string is not properly formatted
    """
    if isinstance(num_list, str):
        try:
            num_list = ast.literal_eval(num_list)
            if not isinstance(num_list, list):
                raise ValueError("String must represent a list")
        except (ValueError, SyntaxError) as e:
            raise ValueError(f"Invalid input string format: {e}")

    # Verify all elements are integers
    if not all(isinstance(x, int) for x in num_list):  # type: ignore
        raise ValueError("All elements must be integers")

    return sum(num_list)

Let’s create an additional `multiply_numbers` tool too.

In [13]:
from typing import List, Union

# import ast # it is already built in Python it seems
from functools import reduce
from operator import mul


def multiply_numbers(num_list: Union[List[int], str]) -> int:
    """
    This function takes either a List of integers or a string representation of a list
    and returns the product of all numbers.

    Args:
        num_list: List[int] or str - Either a list of integers or a string representing a list
            e.g. "[1, 2, 3]" or [1, 2, 3]

    Returns:
        int: The product of all numbers in the list

    Raises:
        ValueError: If the string cannot be converted to a list of integers,
                   if the list is empty, or if any element is not an integer
        SyntaxError: If the string is not properly formatted
    """
    # Handle string input
    if isinstance(num_list, str):
        try:
            num_list = ast.literal_eval(num_list)
            if not isinstance(num_list, list):
                raise ValueError("String must represent a list")
        except (ValueError, SyntaxError) as e:
            raise ValueError(f"Invalid input string format: {e}")

    # Check if list is empty
    if not num_list:
        raise ValueError("List cannot be empty")

    # Verify all elements are integers
    if not all(isinstance(x, int) for x in num_list):  # type: ignore
        raise ValueError("All elements must be integers")

    # Calculate product using reduce and multiplication operator
    return reduce(mul, num_list)


Time to use this tool with our LLM to see how well it works.

In [14]:
tools = [add_numbers, multiply_numbers]
tool_schemas = [function_to_schema(tool) for tool in tools]

response = run_llm(
    content="""
    [23,51,321]
    """,
    system_message="""
    Use the appropriate tool to calculate the sum of numbers, and only the tool and nothing else.
    """,
    tool_schemas=tool_schemas,
)

print(response)

# [ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_enJSWBrayTgSlFCKgrgw6BGz', function=Function(arguments='{"num_list":"[23,51,321]"}', name='add_numbers'), type='function')])]


[ChatCompletionMessage(content='[42,2,13]', role='assistant', function_call=None, tool_calls=None, refusal=None), ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CpMJR0JNSkniWMTPekUeI18s', function=Function(arguments='{"num_list": "[23,51,321]"}', name='add_numbers'), type='function', index=0)], refusal=None)]


In [15]:
response

[ChatCompletionMessage(content='[42,2,13]', role='assistant', function_call=None, tool_calls=None, refusal=None),
 ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CpMJR0JNSkniWMTPekUeI18s', function=Function(arguments='{"num_list": "[23,51,321]"}', name='add_numbers'), type='function', index=0)], refusal=None)]

> **Now that we can get the tool names and arguments, its time to create another utility function that can take these info and ACTUALLY EXECUTE THEM**

In [16]:
from openai.types.chat.chat_completion_message_tool_call import (
    ChatCompletionMessageToolCall,
)

In [17]:
type(tools), type(tools[1]), tools[0], tools[1]

(list,
 function,
 <function __main__.add_numbers(num_list: Union[List[int], str]) -> int>,
 <function __main__.multiply_numbers(num_list: Union[List[int], str]) -> int>)

In [18]:
tools_map = {tool.__name__: tool for tool in tools}
messages: List[Dict[str, Any]] = []


def execute_tool_call(
    tool_call: ChatCompletionMessageToolCall, tools_map: Dict[str, Callable[..., int]]
) -> int:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")

    # call corresponding function with provided arguments
    return tools_map[name](**args)


# NOTE: Had to shift from response[0] to response[1]
for tool_call in response[1].tool_calls:  # type:ignore
    result = execute_tool_call(tool_call, tools_map)

    # add result back to conversation
    result_message: Dict[str, Any] = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": result,
    }
    messages.append(result_message)

# Assistant: add_numbers({'num_list': '[23,51,321]'})


Assistant: add_numbers({'num_list': '[23,51,321]'})


**Now we would like our llms to take this response and send an output to the user. Let’s do that.**

In [19]:
from typing import Iterable
from openai.types.chat import ChatCompletionMessageParam

In [20]:
tools

[<function __main__.add_numbers(num_list: Union[List[int], str]) -> int>,
 <function __main__.multiply_numbers(num_list: Union[List[int], str]) -> int>]

In [21]:
# from openai.types.chat import ChatCompletionMessageParam

messages: list[ChatCompletionMessageParam] = [
    {
        "role": "user",
        "content": "Say this is a test",
    },
]


In [22]:
messages.append(
    {
        "role": "assistant",
        "content": "This is a test2",
    }
)

In [23]:
messages


[{'role': 'user', 'content': 'Say this is a test'},
 {'role': 'assistant', 'content': 'This is a test2'}]

In [24]:
api_messages: List[ChatCompletionMessageParam] = [
    {"role": "system", "content": "jo tjere pasda"}
]
api_messages

[{'role': 'system', 'content': 'jo tjere pasda'}]

In [25]:
api_messages: List[ChatCompletionMessageParam] = []
api_messages.append({"role": "user", "content": "asdsaasd"})  # works
api_messages.append({"role": "system", "content": "asdsadsasdsaasd"})  # works
api_messages.extend([{"role": "system", "content": "a"}])  # works
api_messages.extend(
    [{"role": "system", "content": "a"}, {"role": "user", "content": "vv"}]
)  # works
api_messages.append(
    {"role": "tool", "content": "hihere", "tool_call_id": "123"}
)  # works but needs all three keys

# api_messages.append({"role": "randomrole", "content": 'asdsaasd'}) # type error!!!
# api_messages.extend([{"role": "susu", "content": "a"}])  # type error

api_messages

[{'role': 'user', 'content': 'asdsaasd'},
 {'role': 'system', 'content': 'asdsadsasdsaasd'},
 {'role': 'system', 'content': 'a'},
 {'role': 'system', 'content': 'a'},
 {'role': 'user', 'content': 'vv'},
 {'role': 'tool', 'content': 'hihere', 'tool_call_id': '123'}]

In [26]:
a = []
if a:
    print("ff")
else:
    print("eee")

eee


In [27]:
response

[ChatCompletionMessage(content='[42,2,13]', role='assistant', function_call=None, tool_calls=None, refusal=None),
 ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_CpMJR0JNSkniWMTPekUeI18s', function=Function(arguments='{"num_list": "[23,51,321]"}', name='add_numbers'), type='function', index=0)], refusal=None)]

In [28]:
tool_schemas: List[ChatCompletionToolParam] = [
    function_to_schema(tool) for tool in tools
]
tools_map: Dict[str, Callable[..., int]] = {tool.__name__: tool for tool in tools}

api_messages: List[ChatCompletionMessageParam] = [
    {"role": "system", "content": "hi yoyo"}
]
# api_messages.extend(user_messages)

# for msg in user_messages:
#     api_messages.append({"role": msg["role"], "content": msg["content"]})
# api_messages.append({"role": "user", "content": user_messages["content"]})

# === 1. get openai completion ===
response: ChatCompletion = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,
    messages=api_messages,
    tools=tool_schemas,
)

In [29]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

In [30]:
response.choices[0].message

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None, refusal=None)

In [31]:
from openai.types.chat import (
    ChatCompletion,
    ChatCompletionMessage,
    ChatCompletionToolParam,
    ChatCompletionMessageParam,
    ChatCompletionMessageToolCall,
)
from openai._types import (
    NotGiven,
    NOT_GIVEN,
)  # important for the way tool call schema is defined


def run_agent(
    system_message: str,
    tools: List[Callable[..., int]],
    # user_messages: List[Dict[
    #     Literal["role", "content"], str
    # ]],  # allowed keys are "role" and "content
    # tools: List[ChatCompletionToolParam],
    # tools: List[Any],
    user_messages: List[ChatCompletionMessageParam],
):
    num_init_messages = len(user_messages)
    user_messages = user_messages.copy()

    # while True:
    # turn python functions into tools and save a reverse map
    tool_schemas: List[ChatCompletionToolParam] = [
        function_to_schema(tool) for tool in tools
    ]
    tools_map: Dict[str, Callable[..., int]] = {tool.__name__: tool for tool in tools}

    # Make variables in correct types to pass into openai client
    api_messages: List[ChatCompletionMessageParam] = [
        {"role": "system", "content": system_message}
    ]
    api_messages.extend(user_messages)
    tools_param: Union[List[ChatCompletionToolParam], NotGiven] = (
        tool_schemas if tool_schemas else NOT_GIVEN
    )

    # === 1. get openai completion ===
    response: ChatCompletion = client.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=api_messages,
        tools=tools_param,
        n=1,
    )
    print(f"OG response = {response}")

    try:
        message: ChatCompletionMessage = response.choices[0].message
        print(f"response = {response}, message = {message}")
    except Exception as e:
        print(e)
        # break
        return

    # Now append the properly formatted message
    # message is of type ChatCompletionMessage, we need ChatCompletionMessageParam
    result_message: ChatCompletionMessageParam = {
        "role": "assistant",
        "content": message.content,
    }
    user_messages.append(result_message)
    # user_messages.append(message)

    if message.content:  # print assistant response
        print("Assistant response is:", message.content)

    if not message.tool_calls:  # if finished handling tool calls, break
        # break
        print(f"no tool call")
        return

    # === 2. handle tool calls ===

    print(
        f"Message has tool calls as : {message.tool_calls}",
    )

    # tool_call is of type ChatCompletionMessageToolCall
    for tool_call in message.tool_calls:
        result = execute_tool_call(tool_call, tools_map)

        result_message: ChatCompletionMessageParam = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result),
        }
        print(f"result message is {result_message}")

        user_messages.append(result_message)

    # ==== 3. return new messages =====
    return user_messages
    # return user_messages[num_init_messages:]


def execute_tool_call(
    tool_call: ChatCompletionMessageToolCall, tools_map: Dict[str, Callable[..., int]]
):
    name: str = tool_call.function.name
    args: Any = json.loads(tool_call.function.arguments)

    print(f"Assistant executing tool call: {name}({args})")

    # call corresponding function with provided arguments
    result = str(tools_map[name](**args))
    print(f"Assistant result: {result}")

    return result



In [32]:
tools = [add_numbers]
messages: List[ChatCompletionMessageParam] = []

system_message = """
    You are an expert number processor and classifier. Your task is to extract and sum only the numbers from any input, ignoring all non-numeric values.

    Rules:
    1. Only process numeric values (integers)
    2. Ignore all non-numeric values (strings, letters, special characters)
    3. Use the add_numbers function to calculate the sum
    4. Format the input properly before passing to add_numbers

    Examples:
    Input: <input>["hello", 42, "pizza", 2, 5]</input>
    Process: Extract numbers [42, 2, 5]
    Output: 49

    Input: <input>[asj,cg,111,42,2]</input>
    Process: Extract numbers [111, 42, 2]
    Output: 155

    Input: <input>[text, more, 100, words, 50]</input>
    Process: Extract numbers [100, 50]
    Output: 150

    For any input, first extract the numbers, then use add_numbers function to calculate their sum.
    Make sure to format the input as a proper list string with square brackets before passing to add_numbers.
    """


# Funny errors while in a while loop
# while True:
user = input("User: ")
messages.append({"role": "user", "content": user})

new_messages = run_agent(system_message, tools, messages)
messages.extend(new_messages)


OG response = ChatCompletion(id='gen-1742327776-8AwuH3AHUrpLFuuuc8cS', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2WQUGprA9dyP6f2E7kvNnfqw', function=Function(arguments='{"num_list":"[1, 3, 990, 1]"}', name='add_numbers'), type='function', index=0)], refusal=None), native_finish_reason='tool_calls')], created=1742327776, model='openai/gpt-4o-mini', object='chat.completion', system_fingerprint='fp_b8bc95a0ac', usage=CompletionUsage(completion_tokens=27, prompt_tokens=417, total_tokens=444), provider='OpenAI')
response = ChatCompletion(id='gen-1742327776-8AwuH3AHUrpLFuuuc8cS', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_2WQUGprA9dyP6f2E7kvNnfqw', function=Function(arguments='{"num_l

In [33]:
new_messages

[{'role': 'user', 'content': '1,3,990,1'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool',
  'tool_call_id': 'call_2WQUGprA9dyP6f2E7kvNnfqw',
  'content': '995'}]

---
## Agent(LLM call + Tools + Pydantic Model)

Let’s first build a model (This is the pydantic model, I will be referring to these as models)


In [34]:
# import pydantic
from pydantic import BaseModel


class Agent(BaseModel):
    name: str = "Agent"
    llm: str = "gpt-4o-mini"
    system_message: str = "You are a helpful Agent"
    tools: List[Callable[..., int]] = []


> Now we can modify the code we wrote earlier to use this model

In [60]:
from openai.types.chat import (
    ChatCompletion,
    ChatCompletionMessage,
    ChatCompletionToolParam,
    ChatCompletionMessageParam,
    ChatCompletionMessageToolCall,
)
from openai._types import (
    NotGiven,
    NOT_GIVEN,
)  # important for the way tool call schema is defined


def run_agent(agent: Agent, messages: List[ChatCompletionMessageParam]):
    num_init_messages = len(messages)
    messages = messages.copy()

    # while True:

    # turn python functions into tools and save a reverse map
    tool_schemas: List[ChatCompletionToolParam] = [
        function_to_schema(tool) for tool in agent.tools
    ]
    tools_map: Dict[str, Callable[..., int]] = {
        tool.__name__: tool for tool in agent.tools
    }

    api_messages: List[ChatCompletionMessageParam] = [
        {"role": "system", "content": agent.system_message}
    ]
    api_messages.extend(messages)
    tools_param: Union[
        List[ChatCompletionToolParam], NotGiven
    ] = tool_schemas if tool_schemas else NOT_GIVEN

    print(f'message to llm = {api_messages}')

    # === 1. get openai completion ===
    response: ChatCompletion = client.chat.completions.create(
        model=agent.llm,
        messages=api_messages,
        tools=tools_param,
    )

    print(f'response is {response}')

    message: ChatCompletionMessage = response.choices[0].message
    result_message: ChatCompletionMessageParam = {
        "role": "assistant",
        "content": message.content,
    }
    messages.append(result_message)

    if message.content:  # print assistant response
        print("Assistant:", message.content)

    if not message.tool_calls:  # if finished handling tool calls, break
        # break
        return

    # === 2. handle tool calls ===

    for tool_call in message.tool_calls:
        result = execute_tool_call(tool_call, tools_map)

        result_message: ChatCompletionMessageParam = {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": str(result),
        }
        print(f"result message is {result_message}")
        messages.append(result_message)

    # ==== 3. return new messages =====
    return messages[num_init_messages:]


def execute_tool_call(
tool_call: ChatCompletionMessageToolCall, tools_map: Dict[str, Callable[..., int]]
):
    name: str = tool_call.function.name
    args: Any = json.loads(tool_call.function.arguments)

    print(f"Assistant: {name}({args})")
    
    # call corresponding function with provided arguments
    result = str(tools_map[name](**args))
    print(f"Assistant result: {result}")

    return result


>> And just as easily we can run multiple agents

In [44]:
calculator_add = Agent(
    name="Addition Calculator",
    system_message="""You are an expert number processor. Extract and sum only the numbers from any input, ignoring non-numeric values.
    Example:
    Input: [text, 100, words, 50]
    Process: Extract numbers [100, 50]
    Output: 150""",
    tools=[add_numbers],
)

calculator_multiply = Agent(
    name="Multiplication Calculator",
    system_message="""You are an expert number processor. Extract and multiply only the numbers from any input, ignoring non-numeric values.
    Example:
    Input: [text, 4, words, 5]
    Process: Extract numbers [4, 5]
    Output: 20""",
    tools=[multiply_numbers],
)

messages: List[ChatCompletionMessageParam] = []
user_query = "[hello, 10, world, 5, test, 2]"
print("User:", user_query)
messages.append({"role": "user", "content": user_query})
response: List[ChatCompletionMessageParam] | None = run_agent(calculator_add, messages)  # Addition calculator

User: [hello, 10, world, 5, test, 2]
Assistant: add_numbers({'num_list': '[10, 5, 2]'})
Assistant result: 17
result message is {'role': 'tool', 'tool_call_id': 'call_4yQc3gG2ClEGDWmekBLrXoNe', 'content': '17'}


In [45]:
response

[{'role': 'assistant', 'content': ''},
 {'role': 'tool',
  'tool_call_id': 'call_4yQc3gG2ClEGDWmekBLrXoNe',
  'content': '17'}]

In [46]:
messages.extend(response)

In [61]:
messages = messages[:-1]


In [62]:
messages


[{'role': 'user', 'content': '[hello, 10, world, 5, test, 2]'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool',
  'tool_call_id': 'call_4yQc3gG2ClEGDWmekBLrXoNe',
  'content': '17'}]

In [63]:
user_query = "Now multiply these numbers"  # implicitly refers to the numbers from previous input
print("User:", user_query)
messages.append({"role": "user", "content": user_query})
messages


User: Now multiply these numbers


[{'role': 'user', 'content': '[hello, 10, world, 5, test, 2]'},
 {'role': 'assistant', 'content': ''},
 {'role': 'tool',
  'tool_call_id': 'call_4yQc3gG2ClEGDWmekBLrXoNe',
  'content': '17'},
 {'role': 'user', 'content': 'Now multiply these numbers'}]

In [64]:
response = run_agent(calculator_multiply, messages)  # Multiplication calculator
# Gives a null chat completion?!


# User: [hello, 10, world, 5, test, 2]
# Assistant: add_numbers({'num_list': '[10, 5, 2]'})
# Assistant: The sum of the numbers extracted from the input is 17.
# User: Now multiply these numbers
# Assistant: multiply_numbers({'num_list': '[10, 5, 2]'})
# Assistant: The product of the numbers extracted from the input is 100.



message to llm = [{'role': 'system', 'content': 'You are an expert number processor. Extract and multiply only the numbers from any input, ignoring non-numeric values.\n    Example:\n    Input: [text, 4, words, 5]\n    Process: Extract numbers [4, 5]\n    Output: 20'}, {'role': 'user', 'content': '[hello, 10, world, 5, test, 2]'}, {'role': 'assistant', 'content': ''}, {'role': 'tool', 'tool_call_id': 'call_4yQc3gG2ClEGDWmekBLrXoNe', 'content': '17'}, {'role': 'user', 'content': 'Now multiply these numbers'}]
response is ChatCompletion(id=None, choices=None, created=None, model=None, object=None, system_fingerprint=None, usage=None, error={'message': 'Provider returned error', 'code': 400, 'metadata': {'raw': '{\n  "error": {\n    "message": "Invalid parameter: messages with role \'tool\' must be a response to a preceeding message with \'tool_calls\'.",\n    "type": "invalid_request_error",\n    "param": "messages.[3].role",\n    "code": null\n  }\n}', 'provider_name': 'OpenAI'}}, user_

TypeError: 'NoneType' object is not subscriptable

---